In [1]:
import pandas as pd
import numpy as np
import time
import sys
import os.path
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
raw_data = pd.read_csv('Stock_data_postPrep.csv',index_col=['Symbols','Date'])

In [3]:
# Get unique stock list from stock data
#stock_list = list(raw_data.index.get_level_values('Symbols').unique())
#stock_list

In [4]:
model_data = raw_data
model_data.head()

Adj Close  Close       High        Low       Open  \
Symbols      Date                                                            
20MICRONS.NS 2015-01-01  29.752979  30.40  31.000000  30.250000  30.450001   
             2015-01-02  30.095530  30.75  30.950001  30.350000  30.400000   
             2015-01-05  30.193401  30.85  31.400000  30.400000  30.900000   
             2015-01-06  29.606169  30.25  30.900000  30.200001  30.850000   
             2015-01-07  29.704042  30.35  30.549999  30.250000  30.250000   

                          Volume       Adj  Adj fact   Adj High    Adj Low  \
Symbols      Date                                                            
20MICRONS.NS 2015-01-01  37604.0  0.978716       1.0  31.000000  30.250000   
             2015-01-02  76443.0  0.978716       1.0  30.950001  30.350000   
             2015-01-05  32024.0  0.978716       1.0  31.400000  30.400000   
             2015-01-06  58283.0  0.978716       1.0  30.900000  30.200001   
             2015-01-07  53013.0  0.978716       1.0  30.549999  30.250000   

                          Adj Open  per_gain  bin_gain  
Symbols      Date                                       
20MICRONS.NS 2015-01-01  30.450001       NaN         0  
             2015-01-02  30.400000  0.011513         0  
             2015-01-05  30.900000  0.003252         0  
             2015-01-06  30.850000 -0.019449         0  
             2015-01-07  30.250000  0.003306         0

In [5]:
cols = model_data.columns.tolist()
#cols[1:] + cols[0] - Shuffling columns
cols = cols[1:-2] + cols[0:1] + cols[-2:]
model_data = model_data[cols]

In [13]:
# Simple moving average feature
def SMA(df,Variable,n):
    MA=pd.Series((df.groupby('Symbols')[Variable].rolling(n).mean()),name= "SMA_" + str(Variable) + str(n))
    MA=MA.droplevel(0)
    MA_PCH=pd.Series(((df[Variable]-MA)/MA)*100,name= "SMA_PCH_" + str(Variable) + str(n))
    df=df.join(MA)
    df=df.join(MA_PCH)
    return df

#Exponential Moving Average  
def EMA(df,Variable, n):  
    EMA = pd.Series(df.groupby('Symbols')[Variable].apply(lambda x : x.ewm(span = n, min_periods = n - 1).mean()), name = 'EMA_' + str(Variable)+ str(n))  
    #EMA=EMA.droplevel(0)
    EMA_PCH = pd.Series(((df[Variable]-EMA)/EMA )*100,name = 'EMA_PCH_' + str(Variable) +  str(n))
    df = df.join(EMA)  
    df = df.join(EMA_PCH)
    return df

#Rate of change
def ROC(df,Variable, n):  
    diff=df.groupby('Symbols')[Variable].diff(n-1)
    shift=df.groupby('Symbols')[Variable].shift(n-1)
    ROC=pd.Series(diff/shift,name='ROC_' + str(Variable) +  str(n))
    df=df.join(ROC)
    return df

#Momentum
def MOM(df,Variable, n):  
    M = pd.Series(df.groupby('Symbols')[Variable].diff(n), name = 'Momentum_' + str(Variable) + str(n))  
    df=df.join(M)
    return df

def MACD(df,Variable, n):
    MA_12=df.groupby('Symbols')[Variable].rolling(12).mean()
    MA_12=MA_12.droplevel(0)
    MA_26=df.groupby('Symbols')[Variable].rolling(26).mean()
    MA_26=MA_26.droplevel(0)
    MACD=pd.Series(MA_12-MA_26,name= 'MACD_' + str(Variable))
    df=df.join(MACD)
    return df
    
def WPR(df,Variable,n):
    WPL=pd.Series((df.groupby('Symbols')['Adj Low'].rolling(n).min()),name= "WPR_" + str('Adj Low') + str(n))
    WPL=WPL.droplevel(0)
    WPH=pd.Series((df.groupby('Symbols')['Adj High'].rolling(n).max()),name= "WPR_" + str('Adj High') + str(n))
    WPH=WPH.droplevel(0)
    WPR_PCH=pd.Series(((WPH - df[Variable])/(WPH-WPL))*100, name= "WPR_PCH_" + str(Variable) + str(n))
    df=df.join(WPR_PCH)
    return df

#Add Technical Indicator
def add_TA(df,variable,period):
    for p in period:
        df=SMA(df,variable,p)
        df=EMA(df,variable,p)
        df=ROC(df,variable,p)
        df=MOM(df,variable,p)
        df=MACD(df,variable,p)
        df=WPR(df, variable,p)
    return df

model_data_ind = add_TA(model_data,'Adj Close',[12])


KeyboardInterrupt: 

In [6]:
n=12
Variable = "Adj Close"
WPL=pd.Series((model_data.groupby('Symbols')['Adj Low'].rolling(n).min()),name= "WPR_" + str('Adj Low') + str(n))
WPL=WPL.droplevel(0)
#WPL = WPL.fillna(value=0) 
WPH=pd.Series((model_data.groupby('Symbols')['Adj High'].rolling(n).max()),name= "WPR_" + str('Adj High') + str(n))
WPH = WPH.droplevel(0)
#WPH=WPH.fillna(value=100000)                 
WPR_PCH=pd.Series(((WPH - model_data["Adj Close"])/(WPH-WPL))*100, name= "WPR_PCH_" + str("Adj Close") + str(n))

In [7]:
WPR_PCH

Symbols       Date      
20MICRONS.NS  2015-01-01          NaN
              2015-01-02          NaN
              2015-01-05          NaN
              2015-01-06          NaN
              2015-01-07          NaN
                              ...    
ZYDUSWELL.NS  2020-02-14    54.930539
              2020-02-17    58.576372
              2020-02-18    53.298611
              2020-02-19    54.097239
              2020-02-20    49.444411
Name: WPR_PCH_Adj Close12, Length: 1593087, dtype: float64

In [ ]:
WPL[1:50]